# Extended Kalman Filter (EKF)

## Requisitos

In [6]:
import pandas as pd
import numpy as np

import scipy
import scipy.linalg
from scipy import stats
from scipy.stats import norm

import sympy
from sympy import init_printing
from sympy.utilities.codegen import codegen
from sympy import Symbol, symbols, Matrix, sin, cos

import webbrowser #hiperlink
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
init_printing(use_latex=True)

## Projeto das estimativas iniciais

### Vetor de estados de entrada

$$ \mathbf{x}_{k-1} = \begin{bmatrix} x \\ y \\ \theta \end{bmatrix} = \begin{bmatrix} \text{Posição X} \\ \text{Posição Y} \\ \text{Heading} \end{bmatrix} $$

### Vetor de controle de entrada

Sua inserção nas equações depende da origem do problema.

Comentário(s): (investigar se é necessário no caso de veículos autônomos)

$$ \mathbf{u}_{k} = \begin{bmatrix} v \\ \psi \end{bmatrix} = \begin{bmatrix} \text{Velocidade} \\ \text{Yaw Rate} \end{bmatrix} $$

## Sistema não-linear dinâmico a ser rastreado

$$ \mathbf{x}_{k} = \mathbf{f}(\mathbf{x}_{k-1}, \mathbf{u}_k) + w_k + \tau_k $$

### Ruído nos vetores de estado e controle

Os ruídos são variáveis aleatórias com normal de média nula, visto que a influência dos ruídos é filtrada pelas covariâncias.

Comentário(s): (verificar se pode ser otimizado, mas aparentemente é a solução estocástica ótima)

$$ w_k = \mathcal{N}(0, Q) $$

$$ \tau_k = \mathcal{N}(0, M) $$

## Sistema linearizado por série de Taylor

O sistema não-linear estocástico pode ser linearizado e descrito por meio de equações cinemáticas:

Comentário(s): (hodometria típica em robótica móvel) (escolhi a mais simples para facilitar o ajuste fino)

$$ \mathbf{x}_k = \begin{bmatrix} x_k \\ y_k \\ \theta_k \end{bmatrix} \implies \mathbf{x}_{k} = \begin{bmatrix} x_{k-1} \\ y_{k-1} \\ \theta_{k-1} \end{bmatrix} +  \begin{bmatrix} v_{k}\cdot\cos(\theta_{k-1})\cdot dt \\ v_{k}\cdot\sin(\theta_{k-1})\cdot dt \\ \psi_{k}\cdot dt \end{bmatrix}$$

Para facilitar as operações foi adotado a seguinte notação:

$$ \mathbf{f}(x_{k-1}, u_k) = \begin{bmatrix} f_1 \\ f_2 \\ f_3 \end{bmatrix} = \begin{bmatrix} x_{k-1} + v_{k}\cdot\cos(\theta_{k-1})\cdot dt \\ y_{k-1} + v_{k}\cdot\sin(\theta_{k-1})\cdot dt \\ \theta_{k-1} + \psi_{k}\cdot dt \end{bmatrix}$$

$\mathbf{F}_k$ é a matriz Jacobiana de $f(x_{k-1}, u_k)$ com relação ao vetor de estados.

$$
\mathbf{F}_k = \frac{\partial f(x_{k-1}, u_k)}{\partial x_{k-1}} \implies \mathbf{F}_k = \begin{bmatrix}
\frac{\partial f_1}{\partial x} & 
\frac{\partial f_1}{\partial y} &
\frac{\partial f_1}{\partial \theta}\\
\frac{\partial f_2}{\partial x} & 
\frac{\partial f_2}{\partial y} &
\frac{\partial f_2}{\partial \theta} \\
\frac{\partial f_3}{\partial x} & 
\frac{\partial f_3}{\partial y} &
\frac{\partial f_3}{\partial \theta}
\end{bmatrix}
$$

$$\mathbf{F}_k =\begin{bmatrix}
1 & 
0 &
-v_{k}\cdot\sin(\theta_{k-1})\cdot dt \\
0 & 
1 &
v_{k}\cdot\cos(\theta_{k-1})\cdot dt \\
0 & 
0 &
1
\end{bmatrix}
$$

$\mathbf{G}_k$ é a matriz Jacobiana de $f(x_{k-1}, u_k)$ com relação ao vetor de controle.

$$\mathbf{G}_k = \frac{\partial f(x_{k-1}, u_{k})}{\partial u_{k}} \implies \mathbf{G}_k = \begin{bmatrix}
\frac{\partial f_1}{\partial v} & \frac{\partial f_1}{\partial \psi} \\
\frac{\partial f_2}{\partial v} & \frac{\partial f_2}{\partial \psi} \\
\frac{\partial f_3}{\partial v} & \frac{\partial f_3}{\partial \psi}
\end{bmatrix}$$

$$\mathbf{G}_k = \begin{bmatrix}
\cos(\theta_{k-1})\cdot dt & 0 \\
\sin(\theta_{k-1})\cdot dt & 0 \\
0 & dt
\end{bmatrix}$$

## Matriz de covariância de ruídos do sistema de posições

$$\mathbf{Q} = \begin{bmatrix}
\sigma_{x}^{2} & 0 & 0\\
0 & \sigma_{y}^{2} & 0\\
0 & 0 & \sigma_{\theta}^{2}
\end{bmatrix}$$

## Matriz de covariância de ruídos dos sinais de controle de movimento

Controle de erro nas variáveis $v$ e $\psi$ convertido em controle de erro nas variáveis $x$, $y$ e $\theta$.

Comentário(s): (pode ser otimizado por ajuste fino) (filtro que atualiza a cada predição também otimiza)

$$\mathbf M = \begin{bmatrix}
\sigma_{v}^{2} & 0 \\
0 & 
\sigma_{\psi}^{2}
\end{bmatrix}
$$

## Matriz de covariância de estado 

Comentário(s): (investigar melhor) (está associado a origem do veículo)

Certeza da origem:

$$\mathbf{P} = \begin{bmatrix}
0 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & 0
\end{bmatrix}$$

Valor arbitrado razoável (o filtro consegue corrigir em execução)

$$\mathbf{P} = \mathbf{Q} $$

Valor arbitrado pelo professor:

$$\mathbf{P} = \begin{bmatrix}
10 & 0 & 0\\
0 & 10 & 0\\
0 & 0 & 10
\end{bmatrix}$$

## Sistema sensorial não-linear para observar $x_k$

Verificar a propagação de erros do sensor quanto a linearidade. Se for linear, então a distribuição de probabilidades será Gaussiana. Caso contrário, a distribuição de probabilidades será diferente (Poisson por exemplo).

Comentário(s): (otimização sensível !!!) (teorema central do limite) (GPS -> linear)

$$ \mathbf{y}_k = \mathbf{h}(\mathbf{x}_k, v_k) $$

$$ \mathbf{z}_{k} = \mathbf{h}(\mathbf{x}_{k|k-1}) $$

### Ruído no sistema sensorial

Os ruídos são variáveis aleatórias com normal de média nula, visto que a influência dos ruídos é filtrada pelas covariâncias.

Comentário(s): (verificar se pode ser otimizado, mas aparentemente é a solução estocástica ótima)

$$ v_k = \mathcal{N}(0, R) $$

## Sistema sensorial linearizado

Válido somente se a propagação de erros do sensor for linear ! (GNSS/GPS ->  linear) (Senão -> outra distribuição)

$$\mathbf{y}_{k} = \begin{bmatrix}
x_{gps,k} \\
y_{gps,k}
\end{bmatrix}$$ 

Para facilitar as operações foi adotado a seguinte notação:

$$ \mathbf{h}(x_{k}) = \begin{bmatrix} h_1 \\ h_2 \end{bmatrix} = \begin{bmatrix}
x_{gps, k} \\
y_{gps, k}
\end{bmatrix}$$

## Esperança do sistema sensorial linearizado

$$\mathbf{z}_{k} = \begin{bmatrix}
x_{gps, k|k-1} \\
y_{gps, k|k-1}
\end{bmatrix}$$

## 

$\mathbf{H}_k$ é a matriz Jacobiana de $h(x_{k})$ com relação ao vetor de estados.

$$\mathbf{H}_k = \frac{\partial h(x_{k})}{\partial x_{k-1}} \implies \mathbf{H}_k = \begin{bmatrix}
\frac{\partial h_1}{\partial x} & 
\frac{\partial h_1}{\partial y} &
\frac{\partial h_1}{\partial \theta}\\
\frac{\partial h_2}{\partial x} & 
\frac{\partial h_2}{\partial y} &
\frac{\partial h_2}{\partial \theta} \\
\end{bmatrix}$$

$$ \mathbf{H}_{k} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0
\end{bmatrix}$$

## Matriz de covariância do sensor

$$ \mathbf{R} = \begin{bmatrix}
\sigma_{zx}^{2} & 0 \\
0 & 
\sigma_{zy}^{2}
\end{bmatrix}$$

## Matriz identidade

$$\mathbf{I} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}$$

# Equações de previsão das esperanças

### Leitura do sistema sensorial

$$ \mathbf{y}_{k} = \begin{bmatrix}
x_{gps,k} \\
y_{gps,k}
\end{bmatrix} $$ 

### Estimativa do vetor de estados

$$ \mathbf{x}_k = \mathbf{x}_{k-1} + \begin{bmatrix} v_{k-1}\cdot\cos(\theta_{k-1})\cdot dt \\ v_{k-1}\cdot\sin(\theta_{k-1})\cdot dt \\ \psi_{k-1}\cdot dt \end{bmatrix} $$

### Estimativa da covariância de erro

$$ \mathbf{P}_{k|k-1} = \mathbf{F}_{k}\mathbf{P}_{k-1|k-1}\mathbf{F}_{k}^{T} + \mathbf{G}_{k}\mathbf{M}_{k-1}\mathbf{G}_{k}^{T} + \mathbf{Q}_{k} $$

### Esperança do sistema sensorial

$$\mathbf{z}_{k} = \begin{bmatrix}
x_{gps, k|k-1} \\
y_{gps, k|k-1}
\end{bmatrix}$$

# Equações de correção das esperanças

### Ganho de Kalman

Método dos mínimos quadrados

$$ \mathbf{K}_{k} = \mathbf{P}_{k|k-1}\mathbf{H}_{k}^{T}(\mathbf{H}_{k}\mathbf{P}_{k|k-1}\mathbf{H}_{k}^{T} + \mathbf{R}_{k})^{-1} $$

### Correção do vetor de estados

$$ \mathbf{x}_{k|k} = \mathbf{x}_{k|k-1} + \mathbf{K}_{k}(\mathbf{y}_{k}-\mathbf{z}_{k}) $$

### Correção da covariância do erro

$$ \mathbf{P}_{k|k} = (\mathbf{I} - \mathbf{K}_{k}\mathbf{H}_{k})\mathbf{P}_{k|k-1} $$

# Conjunto de dados

Fonte: https://www.tu-chemnitz.de/projekt/smartLoc/gnss_dataset.html.en#Datasets

### Berlin Potsdamer Platz (~49% NLOS)

In [7]:
data1 = "DATASET1.csv"
data1 = pd.read_csv(data1, sep=',')
#data1

### Berlin Gendarmenmarkt (~37% NLOS)

In [8]:
#vou fazer depois
#data2 = "DATASET2.csv"
#data2 = pd.read_csv(data2, sep=',')
#data2

### Simulação

In [9]:
#vou fazer depois

# Implementação do EKF em Python

In [10]:
class EKF:
    def __init__(self):
        # Taxa de amostragem das medições
        self.__dt = 1.0/50.0 # 50 Hz (CAN) (VELOCITY.csv e YAWRATE.csv)
        self.__dtGPS = 1.0/5.0 # 5 Hz (sensor GPS de baixo custo) (NAV-POSLLH.csv)
        
        self.__x = None
        self.__F = None
        
        self.__u = None
        self.__G = None
        
        self.__Q = None
        self.__M = None
        self.__P = np.diag([10.0, 10.0, 10.0])
        
        self.__y = None
        self.__z = None
        self.__H = None
        self.__R = None
        self.__I = np.eye(3)
        # Desvios padrões das variáveis de estado
        self.__noise_pxy = 0.5*2.0*self.__dt**2
        self.__noise_tht = 0.01*self.__dt
        # Desvios padrões dos sinais de controle
        self.__noise_vel = 2.0*self.__dt
        self.__noise_psi = 0.1*self.__dt
        # Desvios padrões das medições
        self.__noise_gps = 4
    
    def init_state(self, psx, psy, tht):
        self.__x = np.matrix([[psx, psy, tht]]).T
    
    def init_control(self, vel, psi):
        self.__u = np.matrix([[vel, psi]]).T
        
    def set_matrices(self, vel, tht):
        f13 = float(-vel*np.sin(tht)*self.__dt)
        f23 = float( vel*np.cos(tht)*self.__dt)
        self.__F = np.matrix([[1.0, 0.0, f13],
                              [0.0, 1.0, f23],
                              [0.0, 0.0, 1.0]])
        g11 = np.cos(tht)
        g21 = np.sin(tht)
        g32 = self.__dt
        self.__G = np.matrix([[g11, 0.0],
                              [g21, 0.0],
                              [0.0, g32]])
        self.__Q = np.diag([self.__noise_pxy**2, self.__noise_pxys**2, self.__noise_tht**2])
        self.__M = np.diag([self.__noise_vel**2, self.__noise_psi**2])
        self.__H = np.matrix([[1.0, 0.0, 0.0],
                              [0.0, 1.0, 0.0]])
        self.__R = np.diag([self.__noise_gps**2, self.__noise_gps**2])
    
    def predict(self):
            self.__P = (self.__F*self.__P*self.__F.T)+(self.__G*self.__M*self.__G.T)+self.__Q